In [1]:
import tensorflow.compat.v2 as tf
import numpy as np
import math

In [2]:
def tf_float32(x):
    """Ensure array/tensor is a float32 tf.Tensor."""
    if isinstance(x, tf.Tensor):
        return tf.cast(x, dtype=tf.float32)  # This is a no-op if x is float32.
    else:
        return tf.convert_to_tensor(x, tf.float32)


In [3]:
class FTM():
    """Synthesize percussive sounds based on physical parameters."""

    def __init__(self,
               n_samples=64000,
               sample_rate=16000,
               tau = 0.3,
               omega = 400,
               p = 0.1,
               D = 0.01,
               alpha=0.5,
               name='ftm'):
        #super().__init__(name=name) #use super to call processor's init class
        self.n_samples = n_samples
        self.sample_rate = sample_rate
        self.tau = tau
        self.omega = omega
        self.p = p
        self.D = D
        self.alpha = alpha

    ## helper function 
    def approxnorm(self,l,mu,s,tau): #take tensor, float, float, int
    # this is a gaussian distribution
        h = l/tau #tensor
        
        i = tf_float32(tf.range(0,tau+1,1))
        x = 1/(s * tf.math.sqrt(2*np.pi) * tf.math.exp(-0.5 * (i * h - mu)**2/s**2))

        return tf_float32(x) #return list
       

    ## calculate excitation function
    #calculate integral and approximate excitation function with gaussian distribution
    def get_gaus_f(self,m,l,tau,r): #takes int, tensor, int, float
        #trapezoid rule to integrate f(x)sin(mpix) from 0 to l
        #(f(a)+f(b))*(b-a)/2
        #integral = 0
        num_m = m
        m = tf.reshape(tf_float32(tf.range(1,m+1,1)),[1,m])
        x = self.approxnorm(l,l*r,0.1,tau) #l is a tensor here
        h = l/tau

        t = tf.reshape(tf_float32(tf.range(0,tau,1)),[1,tau]) #to replace the i
        # m by tau
        integral = (tf_float32(x[:-1])*tf.sin(tf.linalg.matrix_transpose(m)*np.pi*t*h/l)+tf_float32(x[1:])*tf.sin(tf.linalg.matrix_transpose(m)*np.pi*(t+1)*h/l))*h/2
        integral_sum = tf.reduce_sum(integral,axis=1)*2/l
        #for i in range(tau):
          #x(i+2)
          #print(x.shape,x[0],x[0,1])
        #    integral = integral + (x[i] * np.sin(m * np.pi * i * h/l) + x[i+1] * np.sin(m * np.pi * (i + 1) * h/l))*h/2
        #integral = integral*2/l
        return tf.reshape(integral_sum,[1,num_m]) #return tensor

    ## calculate omega, sigma, K
    def getsigma(self,m1,m2,alpha,p,s11):  
        m1 = tf.reshape(tf_float32(tf.range(1,m1+1,1)),[1,m1])
        m2 = tf.reshape(tf_float32(tf.range(1,m2+1,1)),[1,m2])
        beta = alpha + 1/alpha

        sigma = s11 * (1 + p * ((tf.linalg.matrix_transpose(m1)*m1 * alpha + tf.linalg.matrix_transpose(m2)*m2/alpha) - beta))
        return sigma

    def getomega(self,m1,m2,alpha,p,D,w11,s11):
        m1 = tf.reshape(tf_float32(tf.range(1,m1+1,1)),[1,m1])
        m2 = tf.reshape(tf_float32(tf.range(1,m2+1,1)),[1,m2])

        interm = tf.linalg.matrix_transpose(m1)*m1 * alpha + tf.linalg.matrix_transpose(m2)*m2/alpha
        beta = alpha + 1/alpha

        omega_sq = D**2 * w11**2 * interm*interm + interm * (s11*s11 * (1 - p * beta)**2/beta + w11**2 * (1 - D**2 * beta**2)/beta) - s11*s11 * (1 - p * beta)**2
        omega = tf.math.sqrt(omega_sq)
        return tf.where(tf.math.is_nan(omega),tf.zeros_like(omega),omega) #check if there's zero in the omega matrix

    #k is dependent on striking position as well
    def get_gaus_k(self,m1,m2,omega,f1,f2, x1,x2,l,alpha): #take int, int, tensor, tensor, tensor, tensor, tensor, tensor, tensor
        #f1,f2 are both tensors
        m1 = tf.reshape(tf_float32(tf.range(1,m1+1,1)),[1,m1])
        m2 = tf.reshape(tf_float32(tf.range(1,m2+1,1)),[1,m2])

        l2 = l * alpha
        if float(x1) > float(l):
            x1 = 0
        if float(x2) > float(l2):
            x2 = 0
          #f should be of m length, thus m by m pointwise multiply, finally pointwise divide
        k = (tf.linalg.matrix_transpose(f1) * f2) * (tf.linalg.matrix_transpose(tf.sin(m1 * np.pi * x1/l)) * tf.sin(m2 * np.pi * x2/l2))/ omega #assuming x1,x2 at center of the surface
        #k is m by m
        #k = np.round(k,10)
        return k #return tensor

    ## calculate the ending sound 
    def getsounds_imp_gaus(self,m1,m2,r1,r2,w11,tau11,p,D,alpha,sr):
        """
        Calculate sound of a FTM 2D rectangular drum of shape {w,tau,p,D,alpha}
        excitation function:(Laplace) spatially gaussian, temporally delta
        modes: m1, m2 along each direction
        excitation position: r1, r2 
        """
        #w11,tau11,p,D,alpha = theta
        l = tf_float32(np.pi) #does this need to be tensor too?
        l2 = l*tf_float32(alpha)
        w11 = tf_float32(w11)
        s11 = -1/tf_float32(tau11)
        p = tf_float32(p)
        D = tf_float32(D)
        alpha = tf_float32(alpha)

        #sigma=tf.zeros([m1,m2])
        #omega=tf.zeros([m1,m2])
        #k=tf.zeros([m1,m2])

        x1 = l*r1 #tensor times float is a tensor?
        x2 = l2*r2
 
        sigma= self.getsigma(m1,m2,alpha,p,s11)
        omega = self.getomega(m1,m2,alpha,p,D,w11,s11)
        k = self.get_gaus_k(m1,m2,omega,self.get_gaus_f(m1,1,300,r1),self.get_gaus_f(m2,alpha,300,r2),x1,x2,l,alpha)
        dur = 2**15
        #allocate a big tensor
        t = tf.reshape(tf_float32(tf.range(0,dur,1)),[1,1,dur])
        y = tf.reduce_sum(tf.reduce_sum(tf.reshape(k,[m1,m2,1]) * tf.math.exp(tf.reshape(sigma,[m1,m2,1]) * t/sr) * tf.sin(tf.reshape(omega,[m1,m2,1]) * t/sr),axis=0),axis=0)
        #y = [] #should this be a tensor or it can be a list of tensors
        #for t in range(dur):
        #    y.append(tf.reduce_sum(tf.reduce_sum(k * tf.math.exp(sigma * t/sr) * tf.sin(omega * t/sr))))


        y = y/max(y)


        return tf_float32(y)


test subfunctions

getsigma √

approxnorm

get_gauss_f

get_omega

get_gauss_k


In [4]:
ftm = FTM()
alpha = tf.Variable(tf.random.uniform(shape=()))
p = tf.Variable(tf.random.uniform(shape=()))
D = tf.Variable(tf.random.uniform(shape=()))
w11 = tf.Variable(tf.random.uniform(shape=()))
tau11 = tf.Variable(tf.random.uniform(shape=()))
#s11 = tf.Variable(-1/tf.random.uniform(shape=()))
l = tf_float32(np.pi)
with tf.GradientTape(persistent=True) as tape:
    s11 = -1/tau11
    sigma= ftm.getsigma(10,10,alpha,p,s11)
    omega = ftm.getomega(10,10,alpha,p,D,w11,s11)
    norm = ftm.approxnorm(l,l*0.5,0.1,300)
    f = ftm.get_gaus_f(10,alpha,300,0.5)
    k = ftm.get_gaus_k(10,10,omega,f,f, l*0.5,l*0.5,l,alpha)
    y = ftm.getsounds_imp_gaus(10,10,0.5,0.5,w11,tau11,p,D,alpha,16000)
print(tape.gradient(sigma,[alpha,p,tau11]))
print(tape.gradient(omega,[alpha,p,D,w11,tau11]))
print(tape.gradient(f,alpha))
print(tape.gradient(k,[alpha,p,D,w11,tau11]))
print(tape.gradient(y,[w11,tau11,p,D,alpha]))

[<tf.Tensor: shape=(), dtype=float32, numpy=789.78784>, <tf.Tensor: shape=(), dtype=float32, numpy=-9033.355>, <tf.Tensor: shape=(), dtype=float32, numpy=11509.004>]
[<tf.Tensor: shape=(), dtype=float32, numpy=-108.988434>, <tf.Tensor: shape=(), dtype=float32, numpy=1239.0825>, <tf.Tensor: shape=(), dtype=float32, numpy=424.8756>, <tf.Tensor: shape=(), dtype=float32, numpy=296.54974>, <tf.Tensor: shape=(), dtype=float32, numpy=-598.71045>]
tf.Tensor(356701.8, shape=(), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=45073772.0>, <tf.Tensor: shape=(), dtype=float32, numpy=1281129.5>, <tf.Tensor: shape=(), dtype=float32, numpy=100014.84>, <tf.Tensor: shape=(), dtype=float32, numpy=-1770456.4>, <tf.Tensor: shape=(), dtype=float32, numpy=-619027.5>]
[<tf.Tensor: shape=(), dtype=float32, numpy=-3424.503>, <tf.Tensor: shape=(), dtype=float32, numpy=10841.517>, <tf.Tensor: shape=(), dtype=float32, numpy=677.55566>, <tf.Tensor: shape=(), dtype=float32, numpy=-20.454315>, <tf.Tensor:

## test sounds

In [5]:
ftm = FTM()

In [6]:
theta = [800,0.3,0.1,0.01,0.5]
y = ftm.getsounds_imp_gaus(m1=10,m2=10,r1=0.3,r2=0.3,w11=800,tau11=0.3,p=0.1,D=0.01,alpha=0.5,sr=16000)

In [7]:
import IPython.display as ipd

In [8]:
ipd.Audio(y,rate=16000)

In [257]:
y

<tf.Tensor: id=8539401, shape=(32768,), dtype=float32, numpy=
array([ 0.0000000e+00,  2.9536262e-03,  5.2662306e-03, ...,
       -2.3940327e-05, -2.4817497e-05, -2.5669373e-05], dtype=float32)>

## test gradient descent

objective: minimize loss in theta, Loss(theta) = || S(target) - S(G(theta)) ||_2
    
     find theta such that is closest to target sound


GradientTape, tf.gradients

In [4]:
import IPython.display as ipd
import numpy as np

from kymatio.tensorflow import Scattering1D


use scattering transform as tf representation - need to use pytorch??
but maybe it's not possible bc the intermediate variables are not torch?

In [5]:
scattering = Scattering1D(J=8, shape=(2**15,), Q=1)
#make target sound as a tensor
theta_true = [800,0.3,0.1,0.01,0.5]
theta_est = np.random.random(5)
ftm = FTM()
y_target= ftm.getsounds_imp_gaus(m1=10,m2=10,r1=0.3,r2=0.3,
                                 w11=theta_true[0],
                                 tau11=theta_true[1],
                                 p=theta_true[2],
                                 D=theta_true[3],
                                 alpha=theta_true[4],sr=16000)


y_est= ftm.getsounds_imp_gaus(m1=10,m2=10,r1=0.3,r2=0.3,
                              w11=theta_est[0],
                                 tau11=theta_est[1],
                                 p=theta_est[2],
                                 D=theta_est[3],
                                 alpha=theta_est[4],sr=16000) 

#y_target = torch.Tensor(y_target.numpy())

In [8]:
w11_est = tf.Variable(tf.random.uniform(shape=()))
tau11_est = tf.Variable(tf.random.uniform(shape=()))
p_est = tf.Variable(tf.random.uniform(shape=()))
D_est = tf.Variable(tf.random.uniform(shape=()))
alpha_est = tf.Variable(tf.random.uniform(shape=()))

sc_target = tf_float32(scattering(y_target))
#print(sc_target)
with tf.GradientTape(persistent=True) as tape:
    y_est= ftm.getsounds_imp_gaus(m1=tf.constant(10,dtype=tf.int32),m2=tf.constant(10,dtype=tf.int32),
                                              r1=tf.constant(0.3,dtype=tf.float32),
                                              r2=tf.constant(0.3,dtype=tf.float32),
                                              w11=w11_est,
                                              tau11=tau11_est,
                                              p=p_est,
                                              D=D_est,
                                              alpha=alpha_est,
                                              sr=16000)

    sc_est = tf_float32(scattering(y_est))

    loss = tf.norm(sc_target - sc_est)
    print(tf.norm(sc_target-sc_est))

tf.Tensor(7.93616, shape=(), dtype=float32)


In [10]:
grad = tape.gradient(loss, [w11_est,tau11_est,p_est,D_est,alpha_est])
#print(tape.gradient(sc_est,[w11_est,tau11_est,p_est,D_est,alpha_est]))
print(grad)

[<tf.Tensor: shape=(), dtype=float32, numpy=-0.76357424>, <tf.Tensor: shape=(), dtype=float32, numpy=3.9844573>, <tf.Tensor: shape=(), dtype=float32, numpy=0.43535995>, <tf.Tensor: shape=(), dtype=float32, numpy=-0.0053822994>, <tf.Tensor: shape=(), dtype=float32, numpy=-1.774162>]


use spectrogram as tf representation - use tensorflow

In [10]:
#obtain spectrogram of target sound
theta_true = [800,0.3,0.1,0.01,0.5]
w11,tau11,p,D,alpha=theta_true
ftm = FTM()
y_target= ftm.getsounds_imp_gaus(m1=10,m2=10,r1=0.3,r2=0.3,w11=w11,tau11=tau11,p=p,D=D,alpha=alpha,sr=16000)
#print(y_target)

In [11]:
s_target = tf.signal.stft(
    y_target, frame_length=512, frame_step=256, fft_length=None,
    window_fn=tf.signal.hann_window, pad_end=False, name=None
)
#log-scaled spectrogram
s_target = 10*np.log10(np.abs(s_target))

In [12]:
w11_est = tf.Variable(tf.random.uniform(shape=()))
tau11_est = tf.Variable(tf.random.uniform(shape=()))
p_est = tf.Variable(tf.random.uniform(shape=()))
D_est = tf.Variable(tf.random.uniform(shape=()))
alpha_est = tf.Variable(tf.random.uniform(shape=()))


with tf.GradientTape(persistent=True) as tape:
    y_est= ftm.getsounds_imp_gaus(m1=tf.constant(10,dtype=tf.int32),m2=tf.constant(10,dtype=tf.int32),
                                              r1=tf.constant(0.3,dtype=tf.float32),
                                              r2=tf.constant(0.3,dtype=tf.float32),
                                              w11=w11_est,
                                              tau11=tau11_est,
                                              p=p_est,
                                              D=D_est,
                                              alpha=alpha_est,
                                              sr=16000)

    #y_est= ftm.getsounds_imp_gaus(m1=10,m2=10,r1=0.3,r2=0.3,theta=list(theta_est.numpy()),sr=16000) 
    stft = tf.math.abs(tf.signal.stft(y_est, frame_length=512, 
                                                             frame_step=256, fft_length=None,
                                                             window_fn=tf.signal.hann_window, 
                                                             pad_end=False, name=None))
    
    s_est = 10*tf.math.log(stft+1e-5)/tf.math.log(tf.constant(10,dtype=tf.float32))
    loss = tf.norm(s_target - s_est)
    print(tf.norm(s_target-s_est))

tf.Tensor(3290.9988, shape=(), dtype=float32)


In [13]:
grad = tape.gradient(loss, [w11_est,tau11_est,p_est,D_est,alpha_est])
#print(tape.gradient(y_est,[w11_est,tau11_est,p_est,D_est,alpha_est]))
print(grad)

W1019 19:39:41.259083 4618767808 deprecation.py:323] From /Users/lilyh/anaconda3/lib/python3.6/site-packages/tensorflow_core/python/ops/array_grad.py:502: _EagerTensorBase.cpu (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.identity instead.


[<tf.Tensor: id=659465, shape=(), dtype=float32, numpy=-3.3862705>, <tf.Tensor: id=659463, shape=(), dtype=float32, numpy=749.036>, <tf.Tensor: id=659516, shape=(), dtype=float32, numpy=-18.56018>, <tf.Tensor: id=659517, shape=(), dtype=float32, numpy=-0.08551645>, <tf.Tensor: id=659588, shape=(), dtype=float32, numpy=15.550793>]
